Preprossin Data

# Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error,confusion_matrix, ConfusionMatrixDisplay,classification_report, precision_recall_curve
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
from sklearn.model_selection import cross_validate, cross_val_score,train_test_split, learning_curve, cross_val_predict
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor
from sklearn.metrics import make_scorer, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, RobustScaler, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn import set_config

In [2]:
df = pd.read_csv('data/raw_data/HDHI-Admission-data.csv')

df.head()

,SNO,MRD No.,D.O.A,D.O.D,AGE,GENDER,RURAL,TYPE OF ADMISSION-EMERGENCY/OPD,month year,DURATION OF STAY,...,CONGENITAL,UTI,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION
0,1,234735,4/1/2017,4/3/2017,81,M,R,E,Apr-17,3,...,0,0,0,0,0,0,0,0,0,0
1,2,234696,4/1/2017,4/5/2017,65,M,R,E,Apr-17,5,...,0,0,0,0,0,0,0,0,0,0
2,3,234882,4/1/2017,4/3/2017,53,M,U,E,Apr-17,3,...,0,0,0,0,0,0,0,0,0,0
3,4,234635,4/1/2017,4/8/2017,67,F,U,E,Apr-17,8,...,0,0,0,0,0,0,0,0,0,0
4,5,234486,4/1/2017,4/23/2017,60,F,U,E,Apr-17,23,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.drop(columns=[
    'SNO',
    'MRD No.',
    'D.O.A',
    'D.O.D',
    'BNP',
    'duration of intensive unit stay'
    ], inplace=True)
df.head()

,AGE,GENDER,RURAL,TYPE OF ADMISSION-EMERGENCY/OPD,month year,DURATION OF STAY,OUTCOME,SMOKING,ALCOHOL,DM,...,CONGENITAL,UTI,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION
0,81,M,R,E,Apr-17,3,DISCHARGE,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,65,M,R,E,Apr-17,5,DISCHARGE,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,53,M,U,E,Apr-17,3,DISCHARGE,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,67,F,U,E,Apr-17,8,DISCHARGE,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,60,F,U,E,Apr-17,23,DISCHARGE,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df = df.replace('EMPTY',np.nan)

In [5]:
from sklearn.impute import SimpleImputer

# Impute missing values
imputer = SimpleImputer(strategy="mean")

# Fit the imputer on the selected columns
imputer.fit(df[['EF', 'GLUCOSE', 'TLC', 'PLATELETS', 'HB', 'CREATININE', 'UREA']])

# Transform the columns and assign the results back to the respective columns
df[['EF', 'GLUCOSE', 'TLC', 'PLATELETS', 'HB', 'CREATININE', 'UREA']] = imputer.transform(df[['EF', 'GLUCOSE', 'TLC', 'PLATELETS', 'HB', 'CREATININE', 'UREA']])

# The mean is stored in the transformer's memory
imputer.statistics_

array([ 43.46125159, 162.90279503,  11.52497691, 238.61702516,
        12.2206348 ,   1.34363595,  50.16118804])

In [6]:
# filter the expiry outcome and drop it

df = df[df['OUTCOME'] == 'DISCHARGE']
df.drop(columns='OUTCOME', inplace=True)

# cleaning 'CHEST INFECTION'

df = df[df['CHEST INFECTION'].isin(['1', '0'])]
df['CHEST INFECTION'] = df['CHEST INFECTION'].astype('int')

df

,AGE,GENDER,RURAL,TYPE OF ADMISSION-EMERGENCY/OPD,month year,DURATION OF STAY,SMOKING,ALCOHOL,DM,HTN,...,CONGENITAL,UTI,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION
0,81,M,R,E,Apr-17,3,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,65,M,R,E,Apr-17,5,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,53,M,U,E,Apr-17,3,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,67,F,U,E,Apr-17,8,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,60,F,U,E,Apr-17,23,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15751,60,F,U,E,Mar-19,9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15752,86,F,U,O,Mar-19,5,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
15754,82,M,U,E,Mar-19,10,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
15755,59,F,U,O,Mar-19,4,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Date feature

In [7]:
month_dict = {
    'Jan':1,
    'Feb':2,
    'Mar':3,
    'Apr':4,
    'May':5,
    'Jun':6,
    'Jul':7,
    'Aug':8,
    'Sep':9,
    'Oct':10,
    'Nov':11,
    'Dec':12
}

df['month_nb'] = df['month year'].apply(lambda x: month_dict[x[:3]])

months_in_a_year = 12

df['sin_admission'] = np.sin(2 * np.pi * (df['month_nb'] - 1) / months_in_a_year)
df['cos_admission'] = np.cos(2 * np.pi * (df['month_nb'] - 1) / months_in_a_year)

df.drop(columns=['month year', 'month_nb'], inplace=True)

df.head()

,AGE,GENDER,RURAL,TYPE OF ADMISSION-EMERGENCY/OPD,DURATION OF STAY,SMOKING,ALCOHOL,DM,HTN,CAD,...,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION,sin_admission,cos_admission
0,81,M,R,E,3,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1.0,6.123234e-17
1,65,M,R,E,5,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1.0,6.123234e-17
2,53,M,U,E,3,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1.0,6.123234e-17
3,67,F,U,E,8,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1.0,6.123234e-17
4,60,F,U,E,23,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1.0,6.123234e-17


## Numerical features

In [8]:
numericals = ['AGE', 'HB', 'TLC', 'PLATELETS', 'GLUCOSE', 'UREA', 'CREATININE', 'EF']

print(df[numericals].isnull().sum())

AGE           0
HB            0
TLC           0
PLATELETS     0
GLUCOSE       0
UREA          0
CREATININE    0
EF            0
dtype: int64


In [9]:
rob = RobustScaler()
numericals = ['AGE', 'HB', 'TLC', 'PLATELETS', 'GLUCOSE', 'UREA', 'CREATININE', 'EF']
df_numerical = df[numericals]
df_numerical = rob.fit_transform(df_numerical)
df_numerical = pd.DataFrame(data=df_numerical, columns=['AGE', 'HB', 'TLC', 'PLATELETS', 'GLUCOSE', 'UREA', 'CREATININE', 'EF'])

df[numericals] = df_numerical.reset_index(drop=True)

df.head()

,AGE,GENDER,RURAL,TYPE OF ADMISSION-EMERGENCY/OPD,DURATION OF STAY,SMOKING,ALCOHOL,DM,HTN,CAD,...,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION,sin_admission,cos_admission
0,1.117647,M,R,E,3,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1.0,6.123234e-17
1,0.176471,M,R,E,5,0,1,0,1,1,...,0,0,0,0,0,0,0,0,1.0,6.123234e-17
2,-0.529412,M,U,E,3,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1.0,6.123234e-17
3,0.294118,F,U,E,8,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1.0,6.123234e-17
4,-0.117647,F,U,E,23,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1.0,6.123234e-17


## Non numerical features

In [10]:
df_non_numerical = df.select_dtypes(exclude=['number'])

enc_binary = OneHotEncoder(drop='if_binary', sparse_output=False)
enc_binary.fit(df_non_numerical)

df_non_numerical[enc_binary.get_feature_names_out()] = enc_binary.transform(df_non_numerical)

df_non_numerical_cols = df_non_numerical.iloc[:, :3].columns

df_non_numerical = df_non_numerical.iloc[:, 3:]

df_non_numerical.columns = df_non_numerical_cols

df_non_numerical

,GENDER,RURAL,TYPE OF ADMISSION-EMERGENCY/OPD
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
15751,0.0,1.0,0.0
15752,0.0,1.0,1.0
15754,1.0,1.0,0.0
15755,0.0,1.0,1.0


In [11]:
df_numerical_reset = df_numerical.reset_index(drop=True)
df_non_numerical_reset = df_non_numerical.reset_index(drop=True)
df_sin_cos = df[['sin_admission', 'cos_admission']].reset_index(drop=True)

treated_cols = list(df_numerical_reset.columns) + list(df_non_numerical_reset.columns) + list(df_sin_cos.columns)

df_other_cols = df.drop(columns=treated_cols)
df_other_cols_reset = df_other_cols.reset_index(drop=True)

df_processed = pd.concat([df_numerical_reset, df_non_numerical_reset, df_sin_cos, df_other_cols_reset], axis=1)
df_processed

,AGE,HB,TLC,PLATELETS,GLUCOSE,UREA,CREATININE,EF,GENDER,RURAL,...,CONGENITAL,UTI,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION
0,1.117647,-0.967742,1.24,0.964602,-0.678571,0.000000,-0.017241,-0.33845,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.176471,0.387097,-0.18,-0.699115,-0.297619,-0.592593,-0.017241,-0.05845,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,-0.529412,-0.612903,0.96,0.893805,0.595238,2.185185,2.396552,0.00000,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,0.294118,0.096774,0.00,0.513274,-0.083333,-0.259259,-0.534483,-0.05845,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,-0.117647,0.354839,-0.16,-1.787611,0.083333,0.777778,0.586207,-1.09845,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13750,-0.117647,-1.419355,3.02,-1.460177,1.011905,-0.592593,-0.706897,-0.21845,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
13751,1.411765,-1.193548,0.76,1.176991,-0.071429,0.851852,0.844828,-0.21845,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
13752,1.176471,-1.032258,0.36,1.274336,0.869048,1.222222,1.706897,-0.45845,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
13753,-0.176471,0.193548,0.52,1.796460,0.190476,-0.185185,-0.189655,0.66155,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0


# Train test split

In [12]:
X = df_processed.drop(columns='DURATION OF STAY')
y = df_processed['DURATION OF STAY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(11004, 49)
(2751, 49)
(11004,)
(2751,)


# Baseline

In [13]:
from sklearn.metrics import mean_absolute_error

y_mean = df['DURATION OF STAY'].mean()

y_pred = [y_mean] * len(df['DURATION OF STAY'])

mae = mean_absolute_error(df['DURATION OF STAY'], y_pred)

print('MAE =', round(mae, 2))

MAE = 3.24


# GridSearch for KNN Model

In [14]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

knn = KNeighborsRegressor()

param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

grid_search = GridSearchCV(
    estimator=knn,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Negative MAE:", grid_search.best_score_)

y_pred = grid_search.best_estimator_.predict(X_test)

mae_test = mean_absolute_error(y_test, y_pred)
print("MAE (Test):", mae_test)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'distance'}
Best Negative MAE: -2.4824213512141475
MAE (Test): 2.540990310272843


# KNN Model with best params

In [15]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


model = KNeighborsRegressor(metric= 'manhattan', n_neighbors= 11, weights= 'distance')

scoring=['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']

cv_results = cross_validate(
    model,
    X_train,
    y_train,
    scoring=scoring,
    cv=5
)

print(f"MAE: {-cv_results['test_neg_mean_absolute_error'].mean()}")
print(f"MSE: {-cv_results['test_neg_mean_squared_error'].mean()}")
print(f"R²: {cv_results['test_r2'].mean()}")

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R² Score: {r2}')

MAE: 2.4824213512141475
MSE: 16.264521533198472
R²: 0.2485003755970067
Mean Absolute Error (MAE): 2.540990310272843
Mean Squared Error (MSE): 21.65065500024745
R² Score: 0.20671053881117096


MAE for KNN Model = 2.48

## Pipelines

In [33]:
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy="mean")),
    ('scaler', RobustScaler())
])

cat_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['AGE', 'HB', 'TLC', 'PLATELETS', 'GLUCOSE', 'UREA', 'CREATININE', 'EF']),
    ('cat_transformer', cat_transformer, ['GENDER', 'RURAL', 'TYPE OF ADMISSION-EMERGENCY/OPD'])
])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn', KNeighborsRegressor(n_neighbors=5))
])

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae:.4f}')


Mean Absolute Error (MAE): 3.1001


# Encapsulating

## Data Cleaning

In [36]:
# Data Cleaning

def data_cleaning(df):
    # drop columns pt1
    df.drop(columns=[
        'SNO',
        'MRD No.',
        'D.O.A',
        'D.O.D',
        'BNP',
        'duration of intensive unit stay'
    ], inplace=True)
    df = df.replace('EMPTY',np.nan)

    # imputer
    imputer = SimpleImputer(strategy="mean")
    imputer.fit(df[['EF', 'GLUCOSE', 'TLC', 'PLATELETS', 'HB', 'CREATININE', 'UREA']])
    df[['EF', 'GLUCOSE', 'TLC', 'PLATELETS', 'HB', 'CREATININE', 'UREA']] = imputer.transform(df[['EF', 'GLUCOSE', 'TLC', 'PLATELETS', 'HB', 'CREATININE', 'UREA']])

    # filter the expiry outcome and drop it
    df = df[df['OUTCOME'] == 'DISCHARGE']
    df.drop(columns='OUTCOME', inplace=True)

    # cleaning 'CHEST INFECTION'
    df = df[df['CHEST INFECTION'].isin(['1', '0'])]
    df['CHEST INFECTION'] = df['CHEST INFECTION'].astype('int')

    # creation of sin and cos features
    month_dict = {
        'Jan':1,
        'Feb':2,
        'Mar':3,
        'Apr':4,
        'May':5,
        'Jun':6,
        'Jul':7,
        'Aug':8,
        'Sep':9,
        'Oct':10,
        'Nov':11,
        'Dec':12
    }
    df['month_nb'] = df['month year'].apply(lambda x: month_dict[x[:3]])
    months_in_a_year = 12
    df['sin_admission'] = np.sin(2 * np.pi * (df['month_nb'] - 1) / months_in_a_year)
    df['cos_admission'] = np.cos(2 * np.pi * (df['month_nb'] - 1) / months_in_a_year)
    df.drop(columns=['month year', 'month_nb'], inplace=True)

    return df


## SK Preprocessing

In [44]:
def sk_learn_proc(df):
    num_transformer = RobustScaler()

    # Encode categorical values
    cat_transformer = OneHotEncoder(drop='if_binary', sparse_output=False)

    preprocessor = ColumnTransformer([
        ('num_transformer', num_transformer, ['AGE', 'HB', 'TLC', 'PLATELETS', 'GLUCOSE', 'UREA', 'CREATININE', 'EF']),
        ('cat_transformer', cat_transformer, ['GENDER', 'RURAL', 'TYPE OF ADMISSION-EMERGENCY/OPD'])],
        remainder='passthrough')

    X = df_processed.drop(columns='DURATION OF STAY')
    y = df_processed['DURATION OF STAY']


    X_proc = pd.DataFrame(preprocessor.fit_transform(X))
    X_proc.columns = list(X.columns)

    return X_proc

In [46]:
df = pd.read_csv('data/raw_data/HDHI-Admission-data.csv')

sk_learn_proc(data_cleaning(df))

,AGE,HB,TLC,PLATELETS,GLUCOSE,UREA,CREATININE,EF,GENDER,RURAL,...,CONGENITAL,UTI,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION
0,1.117647,-0.967742,1.24,0.964602,-0.678571,0.000000,-0.017241,-0.33845,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.176471,0.387097,-0.18,-0.699115,-0.297619,-0.592593,-0.017241,-0.05845,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.529412,-0.612903,0.96,0.893805,0.595238,2.185185,2.396552,0.00000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.294118,0.096774,0.00,0.513274,-0.083333,-0.259259,-0.534483,-0.05845,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.117647,0.354839,-0.16,-1.787611,0.083333,0.777778,0.586207,-1.09845,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13750,-0.117647,-1.419355,3.02,-1.460177,1.011905,-0.592593,-0.706897,-0.21845,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13751,1.411765,-1.193548,0.76,1.176991,-0.071429,0.851852,0.844828,-0.21845,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13752,1.176471,-1.032258,0.36,1.274336,0.869048,1.222222,1.706897,-0.45845,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13753,-0.176471,0.193548,0.52,1.796460,0.190476,-0.185185,-0.189655,0.66155,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
num_transformer = RobustScaler()

# Encode categorical values
cat_transformer = OneHotEncoder(drop='if_binary', sparse_output=False)

preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['AGE', 'HB', 'TLC', 'PLATELETS', 'GLUCOSE', 'UREA', 'CREATININE', 'EF']),
    ('cat_transformer', cat_transformer, ['GENDER', 'RURAL', 'TYPE OF ADMISSION-EMERGENCY/OPD'])],
    remainder='passthrough')
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num_transformer', RobustScaler(),
                                 ['AGE', 'HB', 'TLC', 'PLATELETS', 'GLUCOSE',
                                  'UREA', 'CREATININE', 'EF']),
                                ('cat_transformer',
                                 OneHotEncoder(drop='if_binary',
                                               sparse_output=False),
                                 ['GENDER', 'RURAL',
                                  'TYPE OF ADMISSION-EMERGENCY/OPD'])])

In [43]:
X = df_processed.drop(columns='DURATION OF STAY')
y = df_processed['DURATION OF STAY']


X_proc = pd.DataFrame(preprocessor.fit_transform(X))
X_proc.columns = list(X.columns)

X_proc

,AGE,HB,TLC,PLATELETS,GLUCOSE,UREA,CREATININE,EF,GENDER,RURAL,...,CONGENITAL,UTI,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,INFECTIVE ENDOCARDITIS,DVT,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION
0,1.117647,-0.967742,1.24,0.964602,-0.678571,0.000000,-0.017241,-0.33845,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.176471,0.387097,-0.18,-0.699115,-0.297619,-0.592593,-0.017241,-0.05845,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.529412,-0.612903,0.96,0.893805,0.595238,2.185185,2.396552,0.00000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.294118,0.096774,0.00,0.513274,-0.083333,-0.259259,-0.534483,-0.05845,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.117647,0.354839,-0.16,-1.787611,0.083333,0.777778,0.586207,-1.09845,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13750,-0.117647,-1.419355,3.02,-1.460177,1.011905,-0.592593,-0.706897,-0.21845,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13751,1.411765,-1.193548,0.76,1.176991,-0.071429,0.851852,0.844828,-0.21845,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13752,1.176471,-1.032258,0.36,1.274336,0.869048,1.222222,1.706897,-0.45845,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13753,-0.176471,0.193548,0.52,1.796460,0.190476,-0.185185,-0.189655,0.66155,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
